In [2]:
import pandas as pd
import numpy as np
import json
import requests
import os

In [10]:
df_cctv = pd.read_csv('../data/cctv/부산_사하구_CCTV정보.csv', encoding='UTF-8')
df_cctv

,번호,관리기관명,소재지도로명주소,소재지지번주소,설치목적구분,카메라대수,카메라화소수,촬영방면정보,보관일수,설치연월,관리기관전화번호,WGS84위도,WGS84경도,데이터기준일자
0,1,부산광역시 사하구청,부산광역시 사하구 하신중앙로17번길_27,부산광역시 사하구 장림동 1143,다목적,1,200,360도 전방면,30,2016-11,051-220-4344,35.080571,128.961338,2023-02-16
1,2,부산광역시 사하구청,부산광역시 사하구 장평로135번길_21,부산광역시 사하구 장림동 608-52,다목적,1,200,360도 전방면,30,2016-11,051-220-4344,35.078774,128.972097,2023-02-16
2,3,부산광역시 사하구청,부산광역시 사하구 장평로71번길_50,부산광역시 사하구 장림동 566-9,다목적,1,200,360도 전방면,30,2016-11,051-220-4344,35.070799,128.970949,2023-02-16
3,4,부산광역시 사하구청,부산광역시 사하구 장평로125번길_125,부산광역시 사하구 장림동 625-23,다목적,1,200,360도 전방면,30,2016-11,051-220-4344,35.078661,128.969920,2023-02-16
4,5,부산광역시 사하구청,부산광역시 사하구 다대로257번길 5,부산광역시 사하구 장림동 241-2,다목적,1,200,360도 전방면,30,2016-11,051-220-4344,35.079671,128.978228,2023-02-16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
651,652,부산광역시 사하구청,부산광역시 사하구 다대로135번길_68,부산광역시 사하구 신평동 26-3,다목적,1,200,360도 전방면,30,2016-11,051-220-4344,35.092524,128.980796,2023-02-16
652,653,부산광역시 사하구청,부산광역시 사하구 신산북로57번길_25,부산광역시 사하구 신평동 461,다목적,1,200,360도 전방면,30,2016-11,051-220-4344,35.092274,128.971034,2023-02-16
653,654,부산광역시 사하구청,부산광역시 사하구 다대로144번길_133,부산광역시 사하구 신평동 161-215,다목적,1,200,360도 전방면,30,2016-11,051-220-4344,35.092197,128.974720,2023-02-16
654,655,부산광역시 사하구청,부산광역시 사하구 하신번영로_179,부산광역시 사하구 신평동 665,다목적,1,200,360도 전방면,30,2016-11,051-220-4344,35.098526,128.960334,2023-02-16


In [12]:
df_cctv.insert(loc=len(df_cctv.columns), column='geometry', value=[np.NaN for k in range(0, len(df_cctv))])
for i in range(0, len(df_cctv)):
    df_cctv['geometry'].at[i] = np.array([df_cctv['WGS84위도'][i] , df_cctv['WGS84경도'][i]]).astype(object)

In [13]:
df_cctv

,번호,관리기관명,소재지도로명주소,소재지지번주소,설치목적구분,카메라대수,카메라화소수,촬영방면정보,보관일수,설치연월,관리기관전화번호,WGS84위도,WGS84경도,데이터기준일자,Point(WGS84),geometry
0,1,부산광역시 사하구청,부산광역시 사하구 하신중앙로17번길_27,부산광역시 사하구 장림동 1143,다목적,1,200,360도 전방면,30,2016-11,051-220-4344,35.080571,128.961338,2023-02-16,NaN,"[35.080571, 128.961338]"
1,2,부산광역시 사하구청,부산광역시 사하구 장평로135번길_21,부산광역시 사하구 장림동 608-52,다목적,1,200,360도 전방면,30,2016-11,051-220-4344,35.078774,128.972097,2023-02-16,NaN,"[35.078774, 128.972097]"
2,3,부산광역시 사하구청,부산광역시 사하구 장평로71번길_50,부산광역시 사하구 장림동 566-9,다목적,1,200,360도 전방면,30,2016-11,051-220-4344,35.070799,128.970949,2023-02-16,NaN,"[35.070799, 128.970949]"
3,4,부산광역시 사하구청,부산광역시 사하구 장평로125번길_125,부산광역시 사하구 장림동 625-23,다목적,1,200,360도 전방면,30,2016-11,051-220-4344,35.078661,128.969920,2023-02-16,NaN,"[35.078661, 128.96992]"
4,5,부산광역시 사하구청,부산광역시 사하구 다대로257번길 5,부산광역시 사하구 장림동 241-2,다목적,1,200,360도 전방면,30,2016-11,051-220-4344,35.079671,128.978228,2023-02-16,NaN,"[35.079671, 128.978228]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
651,652,부산광역시 사하구청,부산광역시 사하구 다대로135번길_68,부산광역시 사하구 신평동 26-3,다목적,1,200,360도 전방면,30,2016-11,051-220-4344,35.092524,128.980796,2023-02-16,NaN,"[35.092524, 128.980796]"
652,653,부산광역시 사하구청,부산광역시 사하구 신산북로57번길_25,부산광역시 사하구 신평동 461,다목적,1,200,360도 전방면,30,2016-11,051-220-4344,35.092274,128.971034,2023-02-16,NaN,"[35.092274, 128.971034]"
653,654,부산광역시 사하구청,부산광역시 사하구 다대로144번길_133,부산광역시 사하구 신평동 161-215,다목적,1,200,360도 전방면,30,2016-11,051-220-4344,35.092197,128.974720,2023-02-16,NaN,"[35.092197, 128.97472]"
654,655,부산광역시 사하구청,부산광역시 사하구 하신번영로_179,부산광역시 사하구 신평동 665,다목적,1,200,360도 전방면,30,2016-11,051-220-4344,35.098526,128.960334,2023-02-16,NaN,"[35.098526, 128.960334]"


In [16]:
df_cctv.to_csv('../data/cctv/사하구cctv정보(lat_lon).csv', encoding='CP949')